In [16]:
import PyPDF2
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_ollama.llms import OllamaLLM
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [17]:
pdf_paths = ["/home/bigdata/eva_1_BDA/PDF/PIMENTON_MAYO.pdf", "/home/bigdata/eva_1_BDA/PDF/recetas.pdf"] 

## Extraer info y dividirla

In [18]:
def extraer_info_pdf(pdf_path):
    #Extrae el texto de un archivo PDF usando PyPDF2.
    text_data = ""
    try:
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text_data += page.extract_text() + " "
    except Exception as e:
        print(f"Se produjo un error al leer el PDF: {e}")
    
    return text_data


def dividir_texto(texto):
    #Divide el texto en fragmentos manejables para la indexación.
    #extrae el texto de los archivos PDF y lo divide en fragmentos de 500 caracteres, con un solapamiento de 50 caracteres,
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return splitter.split_text(texto)


def extraer_texto_y_dividir_pdf(pdf_path):
    #Extrae el texto de un archivo PDF y lo divide en fragmentos.
    texto = extraer_info_pdf(pdf_path)
    if not texto:
        print("El texto del PDF estaba vacío")
        texto = []
    
    return dividir_texto(texto)

def extraer_y_dividir_texto_de_pdfs(pdf_paths):
    #Une los dos pdf divididos
    all_chunks = []
    for pdf_path in pdf_paths:
        chunks = extraer_texto_y_dividir_pdf(pdf_path)
        all_chunks.extend(chunks)  # Unir todos los fragmentos de los diferentes PDFs
    return all_chunks

## Crear embeddings y vectorstore

In [19]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

chunks = extraer_y_dividir_texto_de_pdfs(pdf_paths)

vectorstore = Chroma.from_texts(
        texts=chunks,
        collection_name="pdf_data",
        embedding=embeddings,
        persist_directory="./vectorstoreCastellano"
    )

## Hacer consultas a Ollama

In [20]:
llm = OllamaLLM(model="llama3.2", server_url="http://localhost:11434")

# Función para realizar una consulta al vector store
def realizar_consulta(vectorstore, consulta):
    retriever = vectorstore.as_retriever()
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    respuesta = qa_chain.run(consulta)
    return respuesta

In [21]:
# Ejemplo de consulta pdf PIMENTON_MAYO
consulta = "¿Resumeme como se hace el pulpo a la gallega?"
respuesta = realizar_consulta(vectorstore, consulta)
print(f"Respuesta: {respuesta}")

/tmp/ipykernel_90009/428516605.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  respuesta = qa_chain.run(consulta)


Respuesta: Claro que sí, te resumiré los pasos para preparar el pulpo a la gallega:

1. Retira el pulpo del fuego y reserva.
2. Corta los tentáculos en trozos de 1 cm y colócalos sobre las patatas picadas (rodajas) en un plato, cubriendo toda la base.
3. Añade sal al gusto.
4. Mezcla pimentón picante y dulce en partes iguales y espolvorea por encima del pulpo.
5. Limpia y trocea las litiruelas (no se menciona qué es esto, pero se supone que es algo a agregar al plato).
6. Prepara la salsa: pica el ajo y la cebolla, pocha en una cazuelita con aceite, agrega coñac y reduce, tritura y cuela.
7. Enfria el pulpo y separa la cabeza del cuerpo.
8. Cuela las patatas (las que se cocinan en el mismo agua que el pulpo) y colócalas sobre el plato con el pulpo.
9. Una vez que estén listos, hornea todas las porciones juntas a 100°C durante unos minutos para calentarlas.

Espero que esto te ayude a preparar un delicioso pulpo a la gallega.


In [24]:
# Ejemplo de consulta pdf recetas
consulta = "¿Resumeme como se hace el cordero asado?"
respuesta = realizar_consulta(vectorstore, consulta)
print(f"Respuesta: {respuesta}")

Respuesta: No puedo proporcionarte una respuesta detallada a esta pregunta sin más contexto, pero puedo resumir los pasos generales para hacer un cordero asado:

1. Preparar el cordero: Se limpia y se pone la carne de la pierna con una mezcla de ajos machacados, vinagre y aceite.
2. Preparar la salsa: Se cocina la salsa con coñac, ajo y cebolla, y luego se tritura y se reserva.
3. Freír las litiruelas: Se limpian y se cortan en cachitos pequeños, luego se fríen en aceite caliente hasta que estén crujientes.
4. Preparar la salsa para el cordero: Se cocina la salsa con harina, pimentón y caldo de carne, y luego se añade el chorizo y las patas del cordero.
5. Asar el cordero: Se coloca en una bandeja de horno con las verduras y se asa a 100ºC durante un tiempo.
6. Saltear la salsa: Se agrega la pasta cocida a la sartén con las verduras y la salsa, y luego se saltea hasta que esté bien combinado.

Espero que esto te sea útil. Si tienes alguna pregunta adicional, no dudes en preguntar.
